# 實作一個簡單的加法器

要怎麼實作 $a_1+a_2+a_3+\cdots+a_n$ 呢？

> 這裡不考慮 Python 特有的方法。

## 實作第一代加法器

我們先關注「兩個數字的加減」就好。首先你肯定會想到接收兩個數字，回傳加好的結果。

In [1]:
def add_v1(a: int, b: int) -> int:
    return a + b

print(f"{add_v1(1, 2)=}")

add_v1(1, 2)=3


但是函式可能略顯單調，我們把它改寫成物件導向的形式吧！

In [2]:
class Adder:
    def __init__(self, a: int, b: int):
        self.a = a
        self.b = b
    
    def add(self) -> int:
        return self.a + self.b
    
print(Adder(1, 2).add())

3


到這裡為止，我們成功實作了 **任意兩個整數** 的加減。但是我們希望它可以加任意多個。所以，我們讓 `Adder` 可以傳入一個陣列：

In [3]:
class Adder:
    def __init__(self, n: list[int]):
        self.n = n
    
    def add(self) -> int:
        return sum(self.n)
    
print(Adder([ 1, 2, 3, 4 ]).add())

10


不過這個設計看起來很醜。這時候我們就要用到 **Factory Pattern**（工廠模式）了！工廠模式可以建構一個 Adder：

In [4]:
from typing import Self


class AdderFactory:
    numbers = list[int]()
    
    def n(self, n: int) -> Self:
        self.numbers.append(n)
        return self

    def build(self) -> Adder:
        return Adder(self.numbers)

print(AdderFactory().n(1).n(2).n(3).n(4).n(5).build().add())

15


到這裡我們實作了任意個數字的總和。但是我們想實作字串的接合。在此之前，我們先實作一個 interface，實作一個加法必備的基本元素；以及一個 get() 函式可以取回內部的數值：

In [5]:
from typing import Generic, ParamSpec
from abc import ABC, abstractmethod

T = ParamSpec('T')


class Getable(ABC, Generic[T]):
    @abstractmethod
    def get(self) -> T:
        pass

    
class Addable(ABC, Generic[T]):
    @abstractmethod
    def add(self, rhs: T) -> Self:
        pass


class BaseAddType(Getable[T], Addable[T], ABC):
    @abstractmethod
    def __init__(self, v: T):
        pass


然後我們實作整數、浮點數以及字串的 Addable 和 Getable class。

In [6]:
class Integer(BaseAddType[int], ABC):
    def __init__(self, v: int):
        self.v = v
        
    def add(self, rhs: Self) -> Self:
        return Integer(self.get() + rhs.get())
    
    def get(self) -> int:
        return self.v


class Float(BaseAddType[float], ABC):
    def __init__(self, v: float):
        self.v = v
        
    def add(self, rhs: Self) -> Self:
        return Float(self.get() + rhs.get())
    
    def get(self) -> float:
        return self.v
    
    
class String(BaseAddType[str], ABC):
    def __init__(self, v: str):
        self.v = v
        
    def add(self, rhs: Self) -> Self:
        return String(self.get() + rhs.get())
    
    def get(self) -> str:
        return self.v

然後把上面的 Adder 改寫成 BaseAddType:

In [7]:
class Adder:
    def __init__(self, n: list[BaseAddType]):
        self.values = n
    
    def add(self) -> BaseAddType:
        if len(self.values) == 0:
            return BaseAddType()  # <-- 注意，這個要修正！
        
        current_value = self.values[0]
        for value_idx in range(1, len(self.values)):
            current_value = current_value.add(
                self.values[value_idx]
            )
        
        return current_value

# 測試一下
print(Adder([Integer(1), Integer(2)]).add().get())

3


但現在，當 `self.values` 沒有任何元素時，我們沒有辦法回傳空值（比如 `0`, `0.0`, `""`)。所以我們最好再多加一個靜態方法，可以回傳這個類型對應的空值：

In [8]:
T = ParamSpec('T')

class ZeroValueGetable(ABC, Generic[T]):
    @classmethod
    def zero(cls) -> Self:
        pass


class BaseAddZeroType(BaseAddType[T], ZeroValueGetable[T], ABC):
    pass


In [9]:
class IntegerZero(ZeroValueGetable[int], Integer, ABC):
    @classmethod
    def zero(cls) -> Self:
        return IntegerZero(0)


class FloatZero(ZeroValueGetable[float], Float, ABC):
    @classmethod
    def zero(cls) -> Self:
        return FloatZero(0.0)
    
    
class StringZero(ZeroValueGetable[str], String, ABC):
    @classmethod
    def zero(cls) -> Self:
        return StringZero("")

In [10]:
from typing import Type

BAZ = ParamSpec('BAZ', bound=BaseAddZeroType)

class Adder(Generic[BAZ]):
    def __init__(self, n: list[BAZ], at: Type[BAZ]):
        self.values = n
        self.at = at
    
    def add(self) -> BAZ:
        current_value = self.at.zero()
        
        for value in self.values:
            current_value = current_value.add(value)
        
        return current_value

Builder 也如法更改。為了簡化，

In [11]:
from typing import Self

class AdderFactory(Generic[BAZ]):
    def __init__(self, at: Type[BAZ]):
        self.at = at
        self.values = list[BAZ]()
    
    def v(self, rhs: T) -> Self: # T 的定義在上面
        self.values.append(self.at(rhs))
        return self
    
    def build(self) -> Adder[BAZ]:
        return Adder(self.values, self.at)


In [13]:
print(AdderFactory(IntegerZero).v(1).build().add().get())
print(AdderFactory(IntegerZero).v(1).v(2).build().add().get())
print(AdderFactory(IntegerZero).build().add().get())

print(AdderFactory(FloatZero).v(1.234).build().add().get())
print(AdderFactory(FloatZero).v(1.345).v(2.456).build().add().get())
print(AdderFactory(FloatZero).build().add().get())

print(AdderFactory(StringZero).v("Hello").build().add().get())
print(AdderFactory(StringZero).v("Hello").v(" ").v("World").build().add().get())
print(AdderFactory(StringZero).build().add().get())

1
3
0
1.234
3.801
0.0
Hello
Hello World
